# Data loading, storage, and file formats

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [7]:
%pwd

u'C:\\Users\\parksejin\\desktop\\pydal\\ch06'

In [6]:
%cd C:/Users/parksejin/desktop/pydal/ch06

C:\Users\parksejin\desktop\pydal\ch06


## 텍스트 파일 이용하는 방법(p222)

** text 데이터를 DataFrame으로 변환시켜주는 함수**
- read_csv
- read_table
- read_fwf
- read_clipboard

In [44]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


파일 내용 살펴보기
윈도우: !type
유닉스: !cat

In [8]:
df = pd.read_csv('ex1.csv')
df

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

쉼표로 이루어진 파일csv를 불러오려면?
1. read_csv 이용
2. read_table('  ',sep=',') <-구분자(sep)를 쉼표로 지정

In [6]:
pd.read_table('ex1.csv', sep=',')

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

In [7]:
!type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [8]:
pd.read_csv('ex2.csv', header=None)


0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo

In [9]:
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

names를 통해 직접 칼럼 이름을 지정 가능

In [10]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ex2.csv', names=names, index_col='message')

a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12

index_col 을 통해 특정 column을 지정하여 색인으로 만들 수 있음.

In [11]:
!type csv_mindex.csv
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

index_col=['A','B']로 지정하면 A색인 후 B색인 형식의 계층적 색인으로 지정가능

In [12]:
list(open('ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_table('ex3.txt', sep='\s+')
result

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

여러 개의 공백문자(\n)를 표현학수 있는 정규표현식 '\s+' 이용하여 처리

In [9]:
!type ex4.csv
pd.read_csv('ex4.csv')

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


# hey!
a                                                   b        c   d    message
# just wanted to make things more difficult for you NaN      NaN NaN      NaN
# who reads CSV files with computers                 anyway? NaN NaN      NaN
1                                                   2        3   4      hello
5                                                   6        7   8      world
9                                                   10       11  12       foo

skiprows=[m,n] <- m-1,n-1번째 줄은 읽지 않고 skip

pandas는 NA, -1, #IND, NULL처럼 비어있는 값을 문자로 인식하여 결측치(NaN)로 처리

In [18]:
!type ex5.csv
result = pd.read_csv('ex5.csv')
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


something  a   b   c   d message
0       one  1   2   3   4     NaN
1       two  5   6 NaN   8   world
2     three  9  10  11  12     foo

In [14]:
pd.isnull(result).sum()

something    0
a            0
b            0
c            1
d            0
message      1
dtype: int64

pd.isnull() <- 결측치 위치? 

In [21]:
result = pd.read_csv('ex5.csv', na_values=[2])
result

something  a   b   c   d message
0       one  1 NaN   3   4     NaN
1       two  5   6 NaN   8   world
2     three  9  10  11  12     foo

na_values 옵션은 리스트[]나 문자열 집합을 받아서 결측 처리

In [18]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ex5.csv', na_values=sentinels)

something  a   b   c   d message
0       one  1   2   3   4     NaN
1       NaN  5   6 NaN   8   world
2     three  9  10  11  12     NaN

- 열마다 다른 NA문자를 사전dict값으로 넘겨 처리 가능
- message에서는 'foo'와 'NA'를 something에서는 'two'를 결측처리하라

### 텍스트 파일을 조금씩(in pieces) 읽어오기(p228)

In [19]:
result = pd.read_csv('ex6.csv')
result

one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
5     1.817480  0.742273  0.419395 -2.251035   Q
6    -0.776764  0.935518 -0.332872 -1.875641   U
7    -0.913135  1.530624 -0.572657  0.477252   K
8     0.358480 -0.497572 -0.367016  0.507702   S
9    -1.740877 -1.160417 -1.637830  2.172201   G
10    0.240564 -0.328249  1.252155  1.072796   8
11    0.764018  1.165476 -0.639544  1.495258   R
12    0.571035 -0.310537  0.582437 -0.298765   1
13    2.317658  0.430710 -1.334216  0.199679   P
14    1.547771 -1.119753 -2.277634  0.329586   J
15   -1.310608  0.401719 -1.000987  1.156708   E
16   -0.088496  0.634712  0.153324  0.415335   B
17   -0.018663 -0.247487 -1.446522  0.750938   A
18   -0.070127 -1.579097  0.120892  0.671432   F
19   -0.194678 -0.492039  2.359605  0.319810   H
20   -0.248618  0.868707 -0.492226 -0.717959   W
21   -1.091549 -0.867110 -0.647760 -0.832562   C
22    0.641404 -0.138822 -0.621963 -0.284839   C
23    1.216408  0.992687  0.165162 -0.069619   V
24   -0.564474  0.792832  0.747053  0.571675   I
25    1.759879 -0.515666 -0.230481  1.362317   S
26    0.126266  0.309281  0.382820 -0.239199   L
27    1.334360 -0.100152 -0.840731 -0.643967   6
28   -0.737620  0.278087 -0.053235 -0.950972   J
29   -1.148486 -0.986292 -0.144963  0.124362   Y
...        ...       ...       ...       ...  ..
9970  0.633495 -0.186524  0.927627  0.143164   4
9971  0.308636 -0.112857  0.762842 -1.072977   1
9972 -1.627051 -0.978151  0.154745 -1.229037   Z
9973  0.314847  0.097989  0.199608  0.955193   P
9974  1.666907  0.992005  0.496128 -0.686391   S
9975  0.010603  0.708540 -1.258711  0.226541   K
9976  0.118693 -0.714455 -0.501342 -0.254764   K
9977  0.302616 -2.011527 -0.628085  0.768827   H
9978 -0.098572  1.769086 -0.215027 -0.053076   A
9979 -0.019058  1.964994  0.738538 -0.883776   F
9980 -0.595349  0.001781 -1.423355 -1.458477   M
9981  1.392170 -1.396560 -1.425306 -0.847535   H
9982 -0.896029 -0.152287  1.924483  0.365184   6
9983 -2.274642 -0.901874  1.500352  0.996541   N
9984 -0.301898  1.019906  1.102160  2.624526   I
9985 -2.548389 -0.585374  1.496201 -0.718815   D
9986 -0.064588  0.759292 -1.568415 -0.420933   E
9987 -0.143365 -1.111760 -1.815581  0.435274   2
9988 -0.070412 -1.055921  0.338017 -0.440763   X
9989  0.649148  0.994273 -1.384227  0.485120   Q
9990 -0.370769  0.404356 -1.051628 -1.050899   8
9991 -0.409980  0.155627 -0.818990  1.277350   W
9992  0.301214 -1.111203  0.668258  0.671922   A
9993  1.821117  0.416445  0.173874  0.505118   X
9994  0.068804  1.322759  0.802346  0.223618   H
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]

In [21]:
pd.read_csv('ex6.csv', nrows=5)

one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q

10000개의 행을 다 읽어오기는 어려우니 nrows=n 를 이용하여 n개의 행만 불러오기

In [22]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

chunksize=n 파일을 n조각으로 나누어 읽으라는 옵션

In [23]:
chunker = pd.read_csv('ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.order(ascending=False)

In [27]:
tot

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
V    328
I    327
U    326
P    324
D    320
A    320
R    318
Y    314
G    308
S    308
N    306
W    305
T    304
B    302
Z    288
C    286
4    171
6    166
7    164
8    162
3    162
5    157
2    152
0    151
9    150
1    146
dtype: float64

In [24]:
tot[:10]

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
dtype: float64

### 데이터를 텍스트 형식으로 기록하기(p230)

In [28]:
data = pd.read_csv('ex5.csv')
data

something  a   b   c   d message
0       one  1   2   3   4     NaN
1       two  5   6 NaN   8   world
2     three  9  10  11  12     foo

In [38]:
data.to_csv('out.csv')
!type out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


DataFrame의 작업파일.to_csv('A.csv') 메서드를 이용하면 csv형식의 데이터를 'A'라는 이름으로 만들 수 있음

In [40]:
data.to_csv('out1.csv', sep='|')
!type out1.csv

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


sep='|' 구분자를 |로 표시

In [42]:
data.to_csv('out2.csv', na_rep='NULL')
!type out2.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


na_rep='NULL' 결측값을 NULL로 표시하라

In [43]:
data.to_csv('out3.csv', index=False, header=False)
!type out3.csv

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


index와 header값을 표시하지 말라

In [44]:
data.to_csv('out4.csv', index=False, columns=['a', 'b', 'c'])
!type out4.csv

a,b,c
1,2,3.0
5,6,
9,10,11.0


index는 표시하지 말고 column은 a,b,c만 표시하라

In [22]:
help(pd.date_range)

Help on function date_range in module pandas.tseries.index:

date_range(start=None, end=None, periods=None, freq='D', tz=None, normalize=False, name=None, closed=None)
    Return a fixed frequency datetime index, with day (calendar) as the default
    frequency
    
    Parameters
    ----------
    start : string or datetime-like, default None
        Left bound for generating dates
    end : string or datetime-like, default None
        Right bound for generating dates
    periods : integer or None, default None
        If None, must specify start and end
    freq : string or DateOffset, default 'D' (calendar daily)
        Frequency strings can have multiples, e.g. '5H'
    tz : string or None
        Time zone name for returning localized DatetimeIndex, for example
    Asia/Hong_Kong
    normalize : bool, default False
        Normalize start/end dates to midnight before generating date range
    name : str, default None
        Name of the resulting index
    closed : string or No

In [35]:
dates = pd.date_range('1/1/2000', periods=7, freq='A')
dates

<class 'pandas.tseries.index.DatetimeIndex'>
[2000-01-01, ..., 2006-01-01]
Length: 7, Freq: AS-JAN, Timezone: None

In [29]:
ts = Series(np.arange(7), index=dates)
ts

2000-01-01 00:00:00    0
2000-01-01 01:00:00    1
2000-01-01 02:00:00    2
2000-01-01 03:00:00    3
2000-01-01 04:00:00    4
2000-01-01 05:00:00    5
2000-01-01 06:00:00    6
Freq: H, dtype: int32

In [51]:
ts.to_csv('tseries.csv')
!type tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [59]:
Series.from_csv('tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

Sereis에서 to_csv를 이용하는 것보다 from_csv메서드를 이용하는게 더 간편

In [58]:
?pd.DataFrame.to_csv


### 수동으로 구분형식 처리하기(p232)

In [60]:
!type ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [37]:
import csv
f = open('ex7.csv')

reader = csv.reader(f)

- 구분자가 한 글자인 파일은 파이썬 내장 csv모듈을 이용해서 처리 가능
- csv.reader()

In [38]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


큰따옴표"" 를 제거한 튜플을 얻을 수 있음

In [39]:
lines = list(csv.reader(open('ex7.csv')))
header, values = lines[0], lines[1:] 
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [40]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [41]:
reader=csv.reader(f,dialect=my_dialect)

In [42]:
reader=csv.reader(f,delimiter='|')

csv.Dialect 속성 및 역할은 p234 표 6-3 참고

In [43]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))
    writer.writerow(('A', 'B', 'C'))

In [45]:
%pwd

u'C:\\Users\\parksejin\\desktop\\pydal\\ch06'

In [44]:
pd.read_csv('mydata.csv')

one;two;three
0         1;2;3
1         4;5;6
2         7;8;9
3         A;B;C

!type mydata.csv

### JSON data (JavaScript Object Notation)

In [6]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

- json의 기본 자료형: 객체(사전), 배열(리스트), 문자열, 숫자, 불리언, null
- 객체의 키는 반드시 문자열("")
- 파이썬 표준 라이브러리 : json
- JSON 문자열을 파이썬 형태로 변환:json.loads

In [7]:
import json
result = json.loads(obj)
result

{u'name': u'Wes',
 u'pet': None,
 u'places_lived': [u'United States', u'Spain', u'Germany'],
 u'siblings': [{u'age': 25, u'name': u'Scott', u'pet': u'Zuko'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

In [8]:
asjson = json.dumps(result)

json.dumps는 파이썬 객체를 JSON형태로 변환

In [9]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

name  age
0  Scott   25
1  Katie   33

JSON객체의 리스트를 DataFrame 형태로 바꾼후 분 가능

### XML and HTML: 웹 내용 긁어오기(p236)
(이 부분부터 잘 안됩니다. 양해 부탁드립니다.)  

lxml(http://lxml.de): HTML과 XML 형식의 데이터를 읽고 쓸 수 있는 라이브러리 중 하나로 큰 파일을 빠르게 처리 가능

In [48]:
from lxml.html import parse
from urllib2 import urlopen

parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))

doc = parsed.getroot()

In [11]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0xbe88d68>,
 <Element a at 0xbe88db8>,
 <Element a at 0xbe88e08>,
 <Element a at 0xbe88e58>,
 <Element a at 0xbe88ea8>]

In [12]:
lnk = links[28]
lnk


<Element a at 0xbea61d8>

In [13]:
lnk.get('href')


'https://autos.yahoo.com/'

In [14]:
lnk.text_content()

'Autos'

URL과 링크 이름을 가져오기위해 각 엘리먼트에 대해 get메서드를 호출하여 url('https://autos.yahoo.com/')을 얻고 text_content메서드를 사용해 링크 이름('Autos')을 가져온다.

In [15]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls

['https://www.yahoo.com/',
 'https://mail.yahoo.com/?.intl=us&.lang=en-US&.src=ym',
 'https://search.yahoo.com/search',
 'http://news.yahoo.com/',
 'http://sports.yahoo.com/',
 'http://finance.yahoo.com/',
 'https://weather.yahoo.com/',
 'https://games.yahoo.com/',
 'https://answers.yahoo.com/',
 'https://screen.yahoo.com/',
 'https://www.flickr.com/',
 'https://mobile.yahoo.com/',
 'http://everything.yahoo.com/',
 'https://www.yahoo.com/politics',
 'https://celebrity.yahoo.com/',
 'https://www.yahoo.com/movies',
 'https://www.yahoo.com/music',
 'https://www.yahoo.com/tv',
 'https://groups.yahoo.com/',
 'https://www.yahoo.com/health',
 'https://www.yahoo.com/style',
 'https://www.yahoo.com/beauty',
 'https://www.yahoo.com/food',
 'https://www.yahoo.com/parenting',
 'https://www.yahoo.com/makers',
 'https://www.yahoo.com/tech',
 'http://shopping.yahoo.com/',
 'https://www.yahoo.com/travel',
 'https://autos.yahoo.com/',
 'https://homes.yahoo.com/own-rent/',
 'https://www.mozilla.org/fire

In [47]:
urls[-10:]

NameError: name 'urls' is not defined

In [125]:
tables = doc.findall('.//table')
tables

[<Element table at 0xc0dd0e8>,
 <Element table at 0xc0dd138>,
 <Element table at 0xc0dd188>]

In [141]:
calls = tables[2]
calls

<Element table at 0xc0dd188>

In [127]:
puts = tables[1]
tables

[<Element table at 0xc0dd0e8>,
 <Element table at 0xc0dd138>,
 <Element table at 0xc0dd188>]

In [128]:
rows = calls.findall('.//tr')

In [142]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [143]:
_unpack(rows[0], kind='th')
_unpack(rows[1], kind='td')

[u'\r\n                        \r\n                            \r\n                            \u2715\r\n                            [modify]\r\n                        \r\n                    ']

In [147]:
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

In [ ]:
call_data = parse_options_data(calls)
put_data = parse_options_data(puts)
call_data[:10]

### lxml.objectify를 이용해 XML 파싱하기(p240)
(죄송하지만 이것도 pass, 선생님들과 함께 해야할 것 같습니다)

In [4]:
%cd mta_perf/Performance_XML_Data

[Error 3] : u'mta_perf/Performance_XML_Data'
C:\Users\Administrator\desktop\pydal\ch06


In [5]:
!head -21 Performance_MNR.xml

'head'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [16]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

IOError: [Errno 2] No such file or directory: 'Performance_MNR.xml'

In [ ]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = DataFrame(data)
perf

In [ ]:
root

In [ ]:
root.get('href')

In [ ]:
root.text

## 이진 데이터(Binary data)(p243)

In [10]:
frame = pd.read_csv('ex1.csv')
frame

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

In [19]:
frame.save('frame_pickle')

In [20]:
pd.read_pickle('frame_pickle')

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

In [ ]:
pickle을 이용하여 데이터를 저장할 수 있음.

### Using HDF5 format(p244)
- hdf는 위계적 데이터 형식의 자료
- 파이썬에서 사용가능한 HDF5 라이브러리: pyTables와 h5py


In [23]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [24]:
store['obj1']

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

### ms 엑셀 파일에서 데이터 읽어오기(p245)

In [22]:
import xlrd
import openpyxl

In [23]:
xls_file=pd.ExcelFile('data.xlsx')

In [24]:
table=xls_file.parse('Sheet1')

In [25]:
table

something  a   b   c   d message
0       one  1   2   3   4     NaN
1       two  5   6 NaN   8   world
2     three  9  10  11  12     foo

### HTML, 웹API와 함께 사용하기(p245)
- 이부분 역시 선생님들과 함께 해야 할 것 같습니다

In [38]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

<Response [200]>

In [67]:
import requests
url = 'https://api.twitter.com/1.1/search/tweets.json?q=python%20pandas'
# parsed = urlopen('https://twitter.com/search?q=python%20pandas'))

# doc = parsed.getroot()
resp = requests.get(url)
resp

<Response [400]>

In [68]:
import json

In [69]:
data=json.loads(resp)

TypeError: expected string or buffer

In [59]:
data

{u'errors': [{u'code': 64,
   u'message': u'The Twitter REST API v1 is no longer active. Please migrate to API v1.1. https://dev.twitter.com/docs/api/1.1/overview.'}]}

## database와 함께 사용하기

In [50]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

pandas에서 제공하는 sqlite3드라이버를 이용하여 SQLite데이터베이스를 이용할 수 있다고 함.

In [51]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

파이썬 SQL드라이버는 테이블에 대해 select쿼리를 수행하면 튜플 리스트를 반환

In [52]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[(u'Atlanta', u'Georgia', 1.25, 6),
 (u'Tallahassee', u'Florida', 2.6, 3),
 (u'Sacramento', u'California', 1.7, 5)]

In [ ]:
튜플 리스트과 위와 같이 반환되었음. 

In [53]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [54]:
DataFrame(rows, columns=zip(*cursor.description)[0])

a           b     c  d
0      Atlanta     Georgia  1.25  6
1  Tallahassee     Florida  2.60  3
2   Sacramento  California  1.70  5

In [55]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

a           b     c  d
0      Atlanta     Georgia  1.25  6
1  Tallahassee     Florida  2.60  3
2   Sacramento  California  1.70  5

pandas.io.sql 모듈의 read_frame 함수를 이용하여 DB에 쿼리를 보낼 수 있음.
select쿼리문과 DB 연결객체(con)만 넘기면 된다고 함